In [ ]:
!pip install hpfrec

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from random import randint
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import datetime
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import warnings
import pandas as pd
import torch.nn.functional as F
from copy import deepcopy
import pandas as pd
import numpy as np
from hpfrec import HPF

torch.manual_seed(2021)

ratings = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/implicitRatings.csv")
#ratings = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data.csv")
ratings = ratings[['userId', 'productId', 'rating']]
ratingsv = ratings[['userId', 'productId', 'rating']].values

#print(len(np.unique(ratingsv[:, 0])), len(np.unique(ratingsv[:, 1])))

train = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")
#train = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data_train.csv")
train = train[["userId","productId","rating"]]

print(train.shape)

(181913, 3)


In [ ]:
class Popularity():

    def __init__(self ,config):
        self.num_users = config['num_users']
        self.num_items = config['num_items']

    def fit(self,train_data):
        column_names = train_data.columns
        userid = column_names[0]
        itemid = column_names[1]
        ratings = column_names[2]
        self.counts_df = train_data.rename(columns = {userid:"UserId",itemid:"productId",ratings:"Count"})
        self.counts_df["Count"] = 1
        self.counts_df['popularity_score'] = self.counts_df.groupby(["productId"])['Count'].transform(sum) / self.num_users
        self.counts_df = self.counts_df[['productId','popularity_score']].drop_duplicates().set_index('productId').to_dict()


    def predict(self,test_data):
        column_names = test_data.columns
        userid = column_names[0]
        itemid = column_names[1]
        ratings = column_names[2]
        self.test_data = test_data.rename(columns = {userid:"UserId",itemid:"productId",ratings:"Count"})
        prediction = self.test_data['productId'].map(self.counts_df['popularity_score']).fillna(1/(self.num_items+1)).values
        return prediction

class Propensity():

    def __init__(self ,config):
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.method = config['propensity_model']
        if self.method == 'poisson':
            self.model = HPF(k=64,check_every=10, ncores=-1, maxiter=150)
        if self.method == 'popularity':
            self.model = Popularity(config)

    def fit(self,train_data):
        column_names = train_data.columns
        userid = column_names[0]
        itemid = column_names[1]
        ratings = column_names[2]
        self.counts_df = train_data.rename(columns = {userid:"UserId",itemid:"ItemId",ratings:"Count"})
        self.counts_df["Count"] = 1
        self.model.fit(self.counts_df)

    def predict(self,test_data):
        column_names = test_data.columns
        userid = column_names[0]
        itemid = column_names[1]
        ratings = column_names[2]
        self.test_data = test_data.rename(columns = {userid:"UserId",itemid:"ItemId",ratings:"Count"})
        if self.method == 'poisson':
            prediction = self.model.predict(self.test_data["UserId"].values,self.test_data["ItemId"].values)
            return 1 - np.exp(-prediction)
        else:
            prediction = self.model.predict(self.test_data)
            return prediction

In [ ]:
config = {"num_users":7373,"num_items":91091,"latent_dim":64,"batch_size":1000,"propensity_model":"poisson","test_ratio":0.0,"prop":True}
propensity = Propensity(config)
propensity.fit(ratings)

**********************************
Hierarchical Poisson Factorization
**********************************

Number of users: 7373
Number of items: 91091
Latent factors to use: 64

Initializing parameters...
Allocating Phi matrix...
Initializing optimization procedure...
Iteration 10 | train llk: -1783241 | train rmse: 0.9927
Iteration 20 | train llk: -1580674 | train rmse: 0.9821
Iteration 30 | train llk: -1562020 | train rmse: 0.9819
Iteration 40 | train llk: -1560169 | train rmse: 0.9819
Iteration 50 | train llk: -1559325 | train rmse: 0.9818
Iteration 60 | train llk: -1558901 | train rmse: 0.9818
Iteration 70 | train llk: -1558702 | train rmse: 0.9818
Iteration 80 | train llk: -1558589 | train rmse: 0.9818
Iteration 90 | train llk: -1558554 | train rmse: 0.9818
Iteration 100 | train llk: -1558525 | train rmse: 0.9818
Iteration 110 | train llk: -1558504 | train rmse: 0.9818
Iteration 120 | train llk: -1558462 | train rmse: 0.9818
Iteration 130 | train llk: -1558448 | train rmse: 0.9818

In [ ]:
popularity = Popularity(config)
popularity.fit(ratings)

In [ ]:
df2 = popularity.predict(train)
df3 = propensity.predict(train)
print(df2.shape)
print(df3.shape)
print(np.mean(1/(1-df2)))
print(np.mean(1/(1-np.nan_to_num(df3))))

(396314,)
(396314,)
1.0000901014930506
1.00049


In [ ]:
from torch.utils.data import DataLoader,Dataset

class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    def __init__(self, user_tensor, item_tensor, target_tensor,prop_tensor):
        """
        args:
            target_tensor: torch.Tensor, the corresponding rating for <user, item> pair
        """
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor
        self.prop_tensor = prop_tensor

    def __getitem__(self, index):
        if self.prop_tensor == None:
            return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index]
        else:
            return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index], self.prop_tensor[index]

    def __len__(self):
        return self.user_tensor.size(0)

class SampleGenerator(object):

    def __init__(self, ratings,config):
        """
        Modified  from: https://github.com/yihong-chen/neural-collaborative-filtering
        Added batching for validation

        TODO: Add a random splitting
        """


        self.batch_size = config['batch_size']
        self.ratings = ratings
        self.n_users = config['num_users']
        self.n_items = config['num_items']
        self.split_rate_test = config['test_ratio']
        self.train_ratings = ratings

        #self.train_ratings, self.test_ratings = self._split(self.ratings)
        self.isprop = config['prop']

        '''if self.isprop == True: #### if we are using  IPS_MF as model
            prop = Propensity(config)
            prop.fit(self.train_ratings)
            scores = prop.predict(self.ratings)

            self.ratings['prop_score'] = scores
            self.train_ratings['prop_score'] = scores[self.train_ratings.index]'''

    def instance_a_train_loader(self, scores,batch_size):
        """instance train loader for one training epoch"""
        users, items, ratings = self.train_ratings['userId'].tolist(), self.train_ratings['productId'].tolist(), self.train_ratings['rating'].tolist()
        if self.isprop:
            dataset = UserItemRatingDataset(user_tensor=torch.LongTensor(users),
                                            item_tensor=torch.LongTensor(items),
                                            target_tensor=torch.FloatTensor(ratings),
                                            prop_tensor=torch.FloatTensor(scores))
        else:
            dataset = UserItemRatingDataset(user_tensor=torch.LongTensor(users),
                                            item_tensor=torch.LongTensor(items),
                                            target_tensor=torch.FloatTensor(ratings))

        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
from torch.autograd import Variable

class MatrixFactorization(torch.nn.Module):
  def __init__(self, config):
        super(MatrixFactorization, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim']
        self.relu = torch.nn.ReLU()

        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)
        self.fc1 = torch.nn.Linear(128,64)
        self.fc2 = torch.nn.Linear(64,32)
        self.fc3 = torch.nn.Linear(32,1)
  
  def forward(self, user_indices, item_indices):
        #user_indices = user_indices.cuda()
        #item_indices = item_indices.cuda()
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        #element_product = torch.bmm(user_embedding.unsqueeze(1), item_embedding.unsqueeze(1).permute(0, 2, 1)).squeeze()
        element_product = torch.cat([user_embedding,item_embedding],dim=1)
        element_product = torch.sigmoid(self.fc1(element_product))
        element_product = torch.sigmoid(self.fc2(element_product))
        element_product = self.fc3(element_product)
        #element_product = torch.mm(user_embedding,item_embedding.T)
        #return self.relu(element_product)
        return torch.sigmoid(element_product).squeeze()

In [ ]:
#np.savetxt("/content/drive/MyDrive/Copy of Epinions/Data-final/popularity_score_train.txt",df2)
#np.savetxt("/content/drive/MyDrive/Copy of Epinions/Data-final/popularity_score_train.txt",df2)
#np.savetxt("/content/drive/MyDrive/Copy of Epinions/Ciao/propensity_score_train.txt",df3)

np.savetxt("/content/drive/MyDrive/Implicit/Epinions/propensity_score_train.txt",df3)

NameError: ignored

In [ ]:
import numpy as np
#df3 = np.loadtxt("/content/drive/MyDrive/Implicit/Epinions/Data/propensity_score_train.txt")
df3 = np.loadtxt("/content/drive/MyDrive/Implicit/Ciao/propensity_score_train.txt")
#df3 = np.loadtxt("/content/drive/MyDrive/Copy of Epinions/Ciao/propensity_score_train.txt")

df3.shape

(396314,)

In [ ]:
df3 = np.nan_to_num(df3)
print(min(df3))
print(max(df3))
print(np.mean(df3))

print(min(1/(1-df3)))
print(max(1/(1-df3)))
print(np.mean(1/(1-df3)))

0.0
0.6933465003967285
0.00046944799408996183
1.0
3.2610095801735035
1.000490133214483


In [ ]:
#config = {"num_users":37952,"num_items":119015,"latent_dim":64,"batch_size":1000,"propensity_model":"poisson","test_ratio":0.0,"prop":True}
config = {"num_users":7373,"num_items":91091,"latent_dim":64,"batch_size":1000,"propensity_model":"poisson","test_ratio":0.0,"prop":True}
#config = {"num_users":7375,"num_items":105114,"latent_dim":64,"batch_size":1000,"propensity_model":"poisson","test_ratio":0.0,"prop":True}
sample_generator = SampleGenerator(train,config)
train_loader = sample_generator.instance_a_train_loader(df3,1000)

In [ ]:
print(min(1/df2))
print(max(1/df2))
a = np.sum(df3)

52.060356652949245
119016.00000000001


In [ ]:
import dill
model = MatrixFactorization(config)
#model = model.cuda()

In [ ]:
min_loss = 1e+5
breakout = 1
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = torch.nn.BCELoss()
for epoch in range(200):
  total_pred_loss = 0
  for batch_num,batch in enumerate(train_loader):
    user,items,ratings,prop_score = batch[0],batch[1], batch[2],batch[3]
    ratings_pred = model.forward(user,items)
    #prop_score = prop_score.cuda()
    #ratings = ratings.cuda()

    loss = loss_fn(ratings_pred, ratings)
    loss = loss/torch.sum(prop_score)
    a = 1/torch.sum(prop_score)
    loss = loss/a
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_pred_loss += loss.item()
  if(round(total_pred_loss/(batch_num+1),2)<round(min_loss/(batch_num+1),2)):
    min_loss = total_pred_loss
    breakout = 0 
  else:
    breakout+=1
  if(breakout==10):
    break
  if(epoch%10==0):
      file = "/content/drive/MyDrive/Implicit/Epinions/Models/IPS_MF_prop_"+str(epoch)+"_"+str(1e-3)
      torch.save(model.state_dict(),f=file)
      print("Model Saved")
  print("Train prediction loss is ", round((total_pred_loss/(batch_num+1)),2)," for epoch ",epoch)


Model Saved
Train prediction loss is  0.08  for epoch  0
Train prediction loss is  0.01  for epoch  1
Train prediction loss is  0.0  for epoch  2
Train prediction loss is  0.0  for epoch  3
Train prediction loss is  0.0  for epoch  4
Train prediction loss is  0.0  for epoch  5
Train prediction loss is  0.0  for epoch  6
Train prediction loss is  0.0  for epoch  7
Train prediction loss is  0.0  for epoch  8
Train prediction loss is  0.0  for epoch  9
Model Saved
Train prediction loss is  0.0  for epoch  10
Train prediction loss is  0.0  for epoch  11


In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Ciao/Models/IPS_MF_prop_"+str(10)+"_"+str(1e-3),map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Ciao/"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
files = os.listdir(path)
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
model.eval()
for f in files:
  if("test" in f):
    if("test" in f and f in ["ratings_data_test_implicit1_1_0.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_5_0.csv","ratings_data_test_implicit1_10_0.csv"]):
      if(f=="test.csv"):
        continue
      print(f)  
      test = pd.read_csv(path+f)
      test = test[["userId","productId","rating"]]
      test_negatives = _sample_negative(list(test["productId"].unique()),test)
      a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
      del test_negatives
      test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
      for row in a.itertuples():
        test_users.append(int(row.userId))
        test_items.append(int(row.productId))
        ratings.append(float(row.rating))
        for i in range(len(row.negative_samples)):
            negative_users.append(int(row.userId))
            negative_items.append(int(row.negative_samples[i]))
            negative_ratings.append(float(0))  # negative samples get 0 rating
      tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
      predr = []
      neg_predr = []
      for i in range(len(tu)):
        user = torch.LongTensor([tu[i]])
        item = torch.LongTensor([ti[i]])
        predr.append(model.forward(user,item).cpu().detach().numpy())
      for i in range(len(nu)):
        user = torch.LongTensor([nu[i]])
        item = torch.LongTensor([ni[i]])
        neg_predr.append(model.forward(user,item).cpu().detach().numpy())
      subjects = [tu,ti,predr,nu,ni,neg_predr]
      test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
      neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
      # the golden set
      test = pd.DataFrame({'user': test_users,
                          'test_item': test_items,
                          'test_score': test_scores})
      # the full set
      full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                        'item':  np.append(neg_items,test_items),
                        'score':  np.append(neg_scores,test_scores)})
      full = pd.merge(full, test, on=['user'], how='left')
      # rank the items according to the scores for each user
      full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
      full.sort_values(['user', 'rank'], inplace=True)
      subjects = full
      print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
      print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))
      del full

ratings_data_test_implicit1_2_0.csv
Hit Ratio is:  0.1
NDCG Value is:  0.06
ratings_data_test_implicit1_5_0.csv
Hit Ratio is:  0.23
NDCG Value is:  0.15
ratings_data_test_implicit1_10_0.csv
Hit Ratio is:  0.15
NDCG Value is:  0.13
ratings_data_test_implicit1_1_0.csv
Hit Ratio is:  0.08
NDCG Value is:  0.05


In [ ]:
  import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Ciao/testImplicit.csv"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
model.eval()
test = pd.read_csv(path)
test = test[["userId","productId","rating"]]
test_negatives = _sample_negative(list(test["productId"].unique()),test)
a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
del test_negatives
test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
for row in a.itertuples():
    test_users.append(int(row.userId))
    test_items.append(int(row.productId))
    ratings.append(float(row.rating))
for i in range(len(row.negative_samples)):
      negative_users.append(int(row.userId))
      negative_items.append(int(row.negative_samples[i]))
      negative_ratings.append(float(0))  # negative samples get 0 rating
tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
predr = []
neg_predr = []
for i in range(len(tu)):
    user = torch.LongTensor([tu[i]])
    item = torch.LongTensor([ti[i]])
    predr.append(model.forward(user,item).cpu().detach().numpy())
for i in range(len(nu)):
    user = torch.LongTensor([nu[i]])
    item = torch.LongTensor([ni[i]])
    neg_predr.append(model.forward(user,item).cpu().detach().numpy())
subjects = [tu,ti,predr,nu,ni,neg_predr]
test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
# the golden set
test = pd.DataFrame({'user': test_users,
                    'test_item': test_items,
                    'test_score': test_scores})
# the full set
full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                  'item':  np.append(neg_items,test_items),
                  'score':  np.append(neg_scores,test_scores)})
full = pd.merge(full, test, on=['user'], how='left')
# rank the items according to the scores for each user
full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
full.sort_values(['user', 'rank'], inplace=True)
subjects = full
print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

Hit Ratio is:  0.84
NDCG Value is:  0.76


In [ ]:
top_k = full[full['rank']<=5]
top_k

user   item     score  test_item  test_score  rank
273134      0     94  0.998980        259  0.99897754   1.0
273135      0     94  0.998980        622  0.99897766   2.0
273136      0     94  0.998980        493    0.998976   3.0
273137      0     94  0.998980        398   0.9989784   4.0
273138      0     94  0.998980        422    0.998978   5.0
...       ...    ...       ...        ...         ...   ...
1224390  7372  22748  0.998921      22748  0.99892104   1.0
1224391  7372  22748  0.998921      22441    0.998882   2.0
1224392  7372  22748  0.998921      63566   0.9988586   3.0
1224393  7372  22441  0.998882      22748  0.99892104   4.0
1224394  7372  22441  0.998882      22441    0.998882   5.0

[27290 rows x 6 columns]

In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Ciao/train_popularity.txt")
x = top_k.groupby("user")
users = list(x.groups.keys())
pops = []
for i in users:
  a = x.get_group(i)
  items = a['item'].values
  for j in items:
    s= train_popularity[j]
    pops.append(s/5)
print(sum(pops)/len(pops))

11.109945034811723
